In [5]:
import os
import csv
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Set your API key
API_KEY = ""  # Replace with your API key

# Function to authenticate using API key
def authenticate_api_key():
    return build("youtube", "v3", developerKey=API_KEY)

# Function to get channel information
def get_channel_info(channel_id):
    youtube = authenticate_api_key()

    try:
        channel_response = youtube.channels().list(
            part="contentDetails,statistics,snippet",
            id=channel_id
        ).execute()

        if "items" in channel_response:
            channel_info = channel_response["items"][0]
            snippet = channel_info.get("snippet", {})
            title = snippet.get("title", "Unknown Channel")
            statistics = channel_info.get("statistics", {})
            video_count = statistics.get("videoCount", 0)

            print(f"Channel: {title}")
            print(f"Channel ID: {channel_id}")
            print(f"Video Count: {video_count}")
            print("")

        else:
            print(f"No information available for channel with ID: {channel_id}")

    except Exception as e:
        print(f"Error fetching channel information for ID {channel_id}: {str(e)}")

# Function to get counts for multiple channels
def display_channel_info(channel_ids):
    for channel_id in channel_ids:
        get_channel_info(channel_id)

# Example: List of channel IDs
channel_ids = ["UCDogdKl7t7NHzQ95aEwkdMw", "UCh5mLn90vUaB1PbRRx_AiaA", "UCjRkTl_HP4zOh3UFaThgRZw"]

# Display counts for multiple channels
display_channel_info(channel_ids)


Channel: Sidemen
Channel ID: UCDogdKl7t7NHzQ95aEwkdMw
Video Count: 337

Channel: MoreSidemen
Channel ID: UCh5mLn90vUaB1PbRRx_AiaA
Video Count: 975

Channel: SidemenReacts
Channel ID: UCjRkTl_HP4zOh3UFaThgRZw
Video Count: 1229



In [6]:
# Function to get video statistics
def get_video_statistics(youtube, video_id):
    video_response = youtube.videos().list(
        part="snippet,contentDetails,statistics",
        id=video_id
    ).execute()

    video_info = video_response["items"][0]["snippet"]
    video_statistics = video_response["items"][0]["statistics"]

    return {
        "Video ID": video_id,
        "Title": video_info["title"],
        "Published At": video_info["publishedAt"],
        "Views": video_statistics["viewCount"],
        "Likes": video_statistics.get("likeCount", 0),
        "Dislikes": video_statistics.get("dislikeCount", 0),
        "Comments": video_statistics.get("commentCount", 0)
    }

# Function to retrieve video comments
def get_video_comments(youtube, video_id):
    try:
        comments_response = youtube.commentThreads().list(
            part="snippet,replies",
            videoId=video_id,
            order="relevance",  # Order comments by relevance (default is time)
            maxResults=100  # Adjust to get more or fewer comments
        ).execute()

        comments_list = []

        for comment_thread in comments_response["items"]:
            comment = comment_thread["snippet"]["topLevelComment"]["snippet"]
            like_count = comment["likeCount"]

            comments_list.append({
                "Video ID": video_id,
                "Comment": comment["textOriginal"],
                "Author": comment["authorDisplayName"],
                "Like Count": like_count
            })

        return comments_list
    except HttpError as e:
        if e.resp.status == 403 and "commentsDisabled" in str(e):
            print(f"Comments are disabled for video {video_id}")
            return []
        else:
            raise

# Function to retrieve all videos from the channel's uploads with pagination
def get_all_videos(youtube, channel_id):
    videos_list = []

    next_page_token = None
    while True:
        search_response = youtube.search().list(
            part="id",
            channelId=channel_id,
            order="date",  # Order videos by date
            type="video",
            maxResults=1500,  # Adjust to get more or fewer videos per page
            pageToken=next_page_token
        ).execute()

        for item in search_response["items"]:
            video_id = item["id"]["videoId"]
            videos_list.append(video_id)

        next_page_token = search_response.get("nextPageToken")
        if not next_page_token:
            break

    return videos_list

# Function to fetch data for a channel and save it to CSV files
def fetch_and_save_channel_data(channel_id, channel_name, youtube):
    # Get all video IDs from the channel's uploads
    video_ids = get_all_videos(youtube, channel_id)

    # Initialize lists to store video statistics and comments
    video_statistics_list = []
    video_comments_list = []

    # Iterate through each video
    for video_id in video_ids:
        # Retrieve video statistics
        video_statistics_list.append(get_video_statistics(youtube, video_id))

        # Retrieve video comments if comments are enabled
        comments_list = get_video_comments(youtube, video_id)
        video_comments_list.extend(comments_list)

    # Save video statistics to CSV file
    video_statistics_csv_path = f"{channel_name}_video_statistics.csv"
    fieldnames_video_statistics = ["Video ID", "Title", "Published At", "Views", "Likes", "Dislikes", "Comments"]

    with open(video_statistics_csv_path, mode="w", newline="", encoding="utf-8") as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames_video_statistics)

        # Write header
        writer.writeheader()

        # Write data
        for video_statistics in video_statistics_list:
            writer.writerow(video_statistics)

    print(f"Video statistics saved to {video_statistics_csv_path}")

    # Save video comments to CSV file
    video_comments_csv_path = f"{channel_name}_video_comments.csv"
    fieldnames_video_comments = ["Video ID", "Comment", "Author", "Like Count"]

    with open(video_comments_csv_path, mode="w", newline="", encoding="utf-8") as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames_video_comments)

        # Write header
        writer.writeheader()

        # Write data
        for video_comment in video_comments_list:
            writer.writerow(video_comment)

    print(f"Video comments saved to {video_comments_csv_path}")

# Example: List of channels and their corresponding channel IDs
channels_data = [
    {"channel_id": "UCDogdKl7t7NHzQ95aEwkdMw", "channel_name": "Sidemen"},
    {"channel_id": "UCh5mLn90vUaB1PbRRx_AiaA", "channel_name": "MoreSidemen"},
    {"channel_id": "UCjRkTl_HP4zOh3UFaThgRZw", "channel_name": "SidemenReacts"}
]

# Authenticate using API key
youtube_instance = authenticate_api_key()

# Fetch and save data for each channel
for channel_data in channels_data:
    fetch_and_save_channel_data(channel_data["channel_id"], channel_data["channel_name"], youtube_instance)


Video statistics saved to Sidemen_video_statistics.csv
Video comments saved to Sidemen_video_comments.csv
Video statistics saved to MoreSidemen_video_statistics.csv
Video comments saved to MoreSidemen_video_comments.csv
Video statistics saved to SidemenReacts_video_statistics.csv
Video comments saved to SidemenReacts_video_comments.csv
